# 1. Resume Dataset

In [ ]:
# ✅ Install dulu fitz (PyMuPDF)
!pip install PyMuPDF

# ✅ Setelah install baru import
import fitz
import pandas as pd
import numpy as np
import re
import nltk
from google.colab import files
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ✅ Download stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# ✅ Inisialisasi
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load Dataset

Untuk keperluan proses training dan testing model, dataset yang digunakan berasal dari kaggle https://www.kaggle.com/datasets/snehaanbhawal/resume-dataset/code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
resume_df = pd.read_csv("/content/drive/MyDrive/dataset_careercompas/resume_cleaned(1).csv")

### Preprocessing (Clean Data)



In [ ]:
nltk.download('punkt_tab')

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

# ✅ Download dulu data yang diperlukan
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# ✅ Inisialisasi
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# ✅ Ganti fungsi preprocess dengan lemmatization
def preprocess_text_lemmatized(text):
    if not isinstance(text, str):
        return "" # Return empty string for non-string input
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)  # ganti dengan spasi supaya tokenisasi rapi
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(tokens)


# ✅ Jalankan fungsi preprocessing
# Apply the function to the 'Resume_str' column
resume_df['clean_resume'] = resume_df['Resume_str'].apply(preprocess_text_lemmatized)

# Print the first few rows of the DataFrame with the new column
print(resume_df[['Resume_str', 'clean_resume']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                          Resume_str  \
0           HR ADMINISTRATOR/MARKETING ASSOCIATE\...   
1           HR SPECIALIST, US HR OPERATIONS      ...   
2           HR DIRECTOR       Summary      Over 2...   
3           HR SPECIALIST       Summary    Dedica...   
4           HR MANAGER         Skill Highlights  ...   

                                        clean_resume  
0  hr administrator marketing associate hr admini...  
1  hr specialist u hr operation summary versatile...  
2  hr director summary year experience recruiting...  
3  hr specialist summary dedicated driven dynamic...  
4  hr manager skill highlight hr skill hr departm...  


# 2. Job Dataset

Untuk keperluan proses training dan testing model, dataset yang digunakan berasal dari kaggle
https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset

Import Data Job Description via URL CSV

In [ ]:
# URL dataset
url_jobdesc = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTUzmvKziuAJBsv2g5w79DwXh9V3pOWeZPd_l3Mo2Og_GJ0LvYpVnudnoFBIXSer-sohyaQzVgZnA14/pub?output=csv"

# Load data
job_df = pd.read_csv(url_jobdesc)

print("\n💼 Job Description Dataset")
print(job_df.head())


💼 Job Description Dataset
                      Job Title                       Role  \
0  Digital Marketing Specialist       Social Media Manager   
1                 Web Developer     Frontend Web Developer   
2            Operations Manager    Quality Control Manager   
3              Network Engineer  Wireless Network Engineer   
4                 Event Manager         Conference Manager   

                                     Job Description  \
0  Social Media Managers oversee an organizations...   
1  Frontend Web Developers design and implement u...   
2  Quality Control Managers establish and enforce...   
3  Wireless Network Engineers design, implement, ...   
4  A Conference Manager coordinates and manages c...   

                                              skills  \
0  Social media platforms (e.g., Facebook, Twitte...   
1  HTML, CSS, JavaScript Frontend frameworks (e.g...   
2  Quality control processes and methodologies St...   
3  Wireless network design and architec

## Preprocessing (Clean Data)

Stemming dan Pembersihan Format Teks pada Dokumen untuk Analisis Teks Lebih Akurat

In [ ]:
def preprocess_job_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'\n|\r|\t', ' ', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return " ".join(tokens)

### Pengecekan dan Penanganan Missing Values pada Kolom 'Job Description

In [ ]:
print("Missing values pada Job Description dataset:")
print(job_df['Job Description'].isnull().sum())

# Drop baris dengan missing pada kolom utama
job_df = job_df.dropna(subset=['Job Description'])

Missing values pada Job Description dataset:
0


### Menampilkan Daftar Kolom pada Dataset Lowongan Pekerjaan

In [ ]:
print("Kolom Job Description dataset:", job_df.columns)

Kolom Job Description dataset: Index(['Job Title', 'Role', 'Job Description', 'skills', 'Responsibilities',
       'Company'],
      dtype='object')


### Pengecekan Nilai yang Hilang (Missing Values) pada Dataset Resume dan Job Description

In [ ]:
print("Missing values pada Resume Job Description :")
print(job_df.isnull().sum())

Missing values pada Resume Job Description :
Job Title           0
Role                0
Job Description     0
skills              0
Responsibilities    0
Company             0
dtype: int64


### Menghitung Jumlah Baris yang Mengandung Missing Values di Dataset Job Description

In [ ]:
print("Jumlah baris dengan missing values di Job Description:", job_df.isnull().any(axis=1).sum())

Jumlah baris dengan missing values di Job Description: 0


###Identifikasi Data Tidak Lengkap pada DataFrame

In [ ]:
print(job_df[job_df.isnull().any(axis=1)])

Empty DataFrame
Columns: [Job Title, Role, Job Description, skills, Responsibilities, Company]
Index: []


###Menampilkan Tipe Data pada Setiap Kolom dalam DataFrame

In [ ]:
print(job_df.dtypes)

Job Title           object
Role                object
Job Description     object
skills              object
Responsibilities    object
Company             object
dtype: object


### Mengetahui Jumlah Data Duplikat pada Dataset

In [ ]:
print("Duplikat di Job Desc:", job_df.duplicated().sum())

Duplikat di Job Desc: 10


### Proses Deteksi dan Penghapusan Duplikat pada Dataset Job Description

In [ ]:
# Cek jumlah duplikat (sudah kamu tahu: 10)
print("Duplikat di Job Desc sebelum dihapus:", job_df.duplicated().sum())

# Hapus duplikat di Job Description dataset
job_df.drop_duplicates(inplace=True)

# Cek ulang setelah hapus
print("Duplikat di Job Desc setelah dihapus:", job_df.duplicated().sum())

# Jangan lupa reset index supaya rapi
job_df.reset_index(drop=True, inplace=True)

# Tampilkan jumlah data setelah penghapusan
print("Jumlah data Job Description setelah hapus duplikat:", len(job_df))

Duplikat di Job Desc sebelum dihapus: 10
Duplikat di Job Desc setelah dihapus: 0
Jumlah data Job Description setelah hapus duplikat: 1989


### Menyimpan Dataset Job Description yang Sudah Dibersihkan ke File CSV

In [ ]:
job_df.to_csv('jobdesc_clean.csv', index=False)

print("✅ File 'jobdesc_clean.csv' berhasil disimpan.")

File 'jobdesc_clean.csv' berhasil disimpan.


# 3. Feature Extraction (TF-IDF)

In [ ]:
def get_top_k_similar_jobs(resume_text, job_df, k=5):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    # Gabungkan teks penting dari job description
    job_df['combined_text'] = (
        job_df['Job Description'] + ' ' +
        job_df['skills'] + ' ' +
        job_df['Responsibilities'] + ' ' +
        job_df['Job Title'] + ' ' +
        job_df['Role']
    )

    # Preprocessing kolom combined_text
    job_df['clean_text'] = job_df['combined_text'].apply(preprocess_job_text)

    # Gabungkan semua job + 1 resume
    all_texts = job_df['clean_text'].tolist() + [resume_text]

    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    cv_vector = tfidf_matrix[-1]
    job_vectors = tfidf_matrix[:-1]

    similarities = cosine_similarity(cv_vector, job_vectors).flatten()

    top_indices = similarities.argsort()[-k:][::-1]

    return top_indices, similarities

In [ ]:
# Ambil satu resume [Uji Coba]
resume_text = resume_df['clean_resume'].iloc[0]

# Panggil fungsi
top_indices, similarities = get_top_k_similar_jobs(resume_text, job_df, k=20)

# Tampilkan hasil
print("\nTop 20 Job Descriptions based on similarity to CV:")
for i in top_indices:
    print(f"Job Title: {job_df.loc[i, 'Job Title']} - Similarity Score: {similarities[i]:.4f}")


Top 20 Job Descriptions based on similarity to CV:
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Service Manager - Similarity Score: 0.2572
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job Title: Customer Success Manager - Similarity Score: 0.2469
Job

# 4. Model (TensorFlow)

1. Persiapan Dataset

In [ ]:
import pandas as pd
import random


pairs = []

for idx, resume_text in enumerate(resume_df['clean_resume']):
    top_indices, _ = get_top_k_similar_jobs(resume_text, job_df, k=5)

    positive_pairs = [(resume_text, job_df.loc[i, 'clean_text'], 1) for i in top_indices]

    negative_indices = random.sample(
        [i for i in range(len(job_df)) if i not in top_indices],
        len(top_indices)
    )
    negative_pairs = [(resume_text, job_df.loc[i, 'clean_text'], 0) for i in negative_indices]

    pairs.extend(positive_pairs + negative_pairs)

random.shuffle(pairs)
df_pairs = pd.DataFrame(pairs, columns=['resume', 'job_desc', 'label'])


 2. Tokenisasi dan Padding

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Gabungkan semua teks
texts = df_pairs['resume'].tolist() + df_pairs['job_desc'].tolist()

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

# Tokenisasi dan padding
max_len = 200
resume_seq = tokenizer.texts_to_sequences(df_pairs['resume'])
job_seq = tokenizer.texts_to_sequences(df_pairs['job_desc'])

resume_pad = pad_sequences(resume_seq, maxlen=max_len, padding='post')
job_pad = pad_sequences(job_seq, maxlen=max_len, padding='post')

labels = df_pairs['label'].values

3. Bangun Model TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

input_resume = layers.Input(shape=(max_len,))
input_job = layers.Input(shape=(max_len,))

embedding = layers.Embedding(input_dim=10000, output_dim=64)

resume_embed = embedding(input_resume)
job_embed = embedding(input_job)

# BiLSTM bisa diganti dengan CNN/GRU
encoded_resume = layers.Bidirectional(layers.LSTM(64))(resume_embed)
encoded_job = layers.Bidirectional(layers.LSTM(64))(job_embed)

# Gabungkan kedua representasi
merged = layers.concatenate([encoded_resume, encoded_job])
dense1 = layers.Dense(64, activation='relu')(merged)
output = layers.Dense(1, activation='sigmoid')(dense1)

model = Model(inputs=[input_resume, input_job], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 200)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 200)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 200, 64)        │        640,000 │ input_layer[0][0],     │
│                           │                        │                │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 128)            │         66,048 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 128)            │         66,048 │ embedding[1][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 256)            │              0 │ bidirectional[0][0],   │
│                           │                        │                │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │         16,448 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │             65 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 788,609 (3.01 MB)

 Trainable params: 788,609 (3.01 MB)

 Non-trainable params: 0 (0.00 B)

4. Latih Model

In [ ]:
model.fit(
    [resume_pad, job_pad],
    labels,
    batch_size=16,
    epochs=10,
    validation_split=0.2
)

Epoch 1/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 526s 416ms/step - accuracy: 0.7281 - loss: 0.5179 - val_accuracy: 0.8010 - val_loss: 0.4204
Epoch 2/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 557s 413ms/step - accuracy: 0.8320 - loss: 0.3716 - val_accuracy: 0.8502 - val_loss: 0.3470
Epoch 3/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 515s 415ms/step - accuracy: 0.8784 - loss: 0.2894 - val_accuracy: 0.8830 - val_loss: 0.2932
Epoch 4/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 565s 417ms/step - accuracy: 0.9135 - loss: 0.2211 - val_accuracy: 0.8993 - val_loss: 0.2804
Epoch 5/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 559s 415ms/step - accuracy: 0.9331 - loss: 0.1795 - val_accuracy: 0.9066 - val_loss: 0.2621
Epoch 6/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 561s 414ms/step - accuracy: 0.9464 - loss: 0.1463 - val_accuracy: 0.9156 - val_loss: 0.2582
Epoch 7/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 565s 416ms/step - accuracy: 0.9554 - loss: 0.1227 - val_accuracy: 0.9134 - val_loss: 0.2588
Epoch 8/10
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 516s 415ms/step - ac

5. Simpan Model

In [ ]:
model.save("jobmatch_model.h5")

In [ ]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)